In [ ]:
pip install backtrader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 6.2 MB/s eta 0:00:00


In [ ]:
#Generating 1,0,-1 signals

import datetime
import numpy as np
import pandas as pd
import yfinance as yf


In [ ]:
data=pd.read_csv('/content/strategy.csv')

In [ ]:
data.head()

,Unnamed: 0,datetime,open,high,low,close,volume,close\open,signal
0,0,2018-01-01,13715.65,13818.55,12750.00,13135.00,6970.441076,close,0
1,1,2018-01-02,13135.00,14750.00,12890.02,14580.00,16359.347340,close,0
2,2,2018-01-03,14576.87,15473.49,14150.00,14749.97,19457.238601,close,0
3,3,2018-01-04,14710.00,15280.00,13918.04,14811.00,18783.876561,close,0
4,4,2018-01-05,14805.01,17118.13,14600.00,16579.00,22383.657333,close,0


In [ ]:
data.datetime = pd.to_datetime(data.datetime,format="%Y-%m-%d",utc=True)

In [ ]:
import backtrader as bt
import datetime
import yfinance as yf
import datetime as dt


class PandasDataFeed(bt.feeds.PandasData):

        lines=('signal',)

        params = (

        ('signal', 'signal'),  # Add your signal column to the data feed

    )

data['datetime'] = pd.to_datetime(data['datetime'])
data.set_index('datetime', inplace=True)
data = PandasDataFeed(dataname=data,
                      fromdate=datetime.datetime(2018, 1, 1),
                      todate=datetime.datetime(2022, 1, 12)
                            )

In [ ]:
class CommInfoFractional(bt.CommissionInfo):
    def getsize(self, price, cash):
        '''Returns fractional size for cash operation @price'''
        return self.p.leverage * (cash / price)

In [ ]:
#Final backtrader Framework. Takes in a dataframe and gives various parameters.
import backtrader.analyzers as btanalyzers
class MyStrategy(bt.Strategy):

    def log(self, txt, dt=None):
      dt = dt or self.datas[0].datetime.date(0)
      print('%s, %s' % (dt.isoformat(), txt))

    def notify(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                self.opsize = order.executed.size
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

                gross_pnl = (order.executed.price - self.buyprice) * \
                    self.opsize


                net_pnl = gross_pnl - self.buycomm - order.executed.comm
                self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                         (gross_pnl, net_pnl))


    def __init__(self):
        self.percent_of_cash_per_trade = 1
        self.in_position = None  # Flag to track position
        self.trades=0
        self.netreturns=0
        self.entry_price=0
        self.profits=0
        self.profitable_trades=0
        self.loss_trades=0
        self.losses=0
        self.sizes=0

    def next(self):
        #self.log(self.broker.cash)
        if self.in_position==None:
            if self.data.signal[0]==1:
                self.sizes = ((self.broker.cash)-0.0000001) / self.data.close[0]
                self.log(self.broker.cash)
                self.buy(size=self.sizes)
                self.log(f'BUY LONG:{self.data.close[0]}')
                self.in_position='Long'
                self.entry_price=self.data.close[0]

            elif self.data.signal[0]==-1 :
                self.sizes = (self.broker.cash) / self.data.close[0]
                self.log(self.broker.cash)
                self.sell(size=self.sizes)
                #self.log(f'SELL SHORT:{self.data.close[0]}')
                self.in_position='Short'
                self.entry_price=self.data.close[0]
        elif self.in_position=='Long':
            if self.data.signal[0]==-1:
                self.log(self.broker.cash)
                self.close(size=self.sizes)
                self.trades+=1
                self.log(f'Trade Number:{self.trades},CLOSING LONG POSITION:{self.data.close[0]}')
                self.sizes=0
                self.in_position = None

        elif self.in_position=='Short':
            if self.data.signal[0]==1:
                self.log(self.broker.cash)
                self.close(size=self.sizes)
                self.trades+=1
                self.log(f'Trade Number:{self.trades},CLOSING SHORT POSITION:{self.data.close[0]}')
                self.in_position = None
                self.sizes=0

def printTradeAnalysis(analyzer):
    '''
    Function to print the Technical Analysis results in a nice format.
    '''
    #Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = (total_won / total_closed) * 100
    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed, total_won, total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

if __name__ == '__main__':
    cerebro = bt.Cerebro()

    cerebro.adddata(data)
    cerebro.broker.addcommissioninfo(CommInfoFractional())
    cerebro.addstrategy(MyStrategy)
    cerebro.broker.set_coc(True)
    #cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
    #cerebro.broker.set_shortcash(True)
    #cerebro.addsizer(bt.sizers.SizerFix,stake=3)
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe', timeframe=bt.TimeFrame.Days,riskfreerate=0.03 )
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown', fund=True)
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='ta')
    cerebro.broker.set_cash(1000.0)
    cerebro.broker.setcommission(commission=0.00)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    thestrats=cerebro.run()
    thestrat = thestrats[0]
    printTradeAnalysis(thestrat.analyzers.ta.get_analysis())
    print('Sharpe Ratio:', thestrat.analyzers.mysharpe.get_analysis())
    print('Max Drawdown:', thestrat.analyzers.drawdown.get_analysis())
    #print('Net Returns:',thestrat.netreturns)
    #print('Profits:',thestrat.profits)
    #print('Losses:',thestrat.losses)
    #print('Profitable Trades:',thestrat.profitable_trades)
    #print('Loss Trades:',thestrat.loss_trades)
    #print('Total Closed Trades:',thestrat.trades)
    #print('Average Winning Trade:' ,thestrat.profits/thestrat.profitable_trades)
    #print('Average Losing Trade:' ,thestrat.losses/thestrat.loss_trades)
    print('')
    print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())
    #print(MyStrategy.returns)


Starting Portfolio Value: 1000.00
2018-03-02, 1000.0
2018-03-02, BUY LONG:10909.99
2018-03-03, BUY EXECUTED, Price: 10909.99, Cost: 1000.00, Comm 0.00
2018-03-14, 9.999996564147295e-08
2018-03-14, Trade Number:1,CLOSING LONG POSITION:8250.17
2018-03-15, SELL EXECUTED, Price: 8250.17, Cost: 1000.00, Comm 0.00
2018-03-15, OPERATION PROFIT, GROSS -243.80, NET -243.80
2018-03-15, 756.2032596057359
2018-03-16, SELL EXECUTED, Price: 8126.00, Cost: -756.20, Comm 0.00
2018-03-16, OPERATION PROFIT, GROSS -255.18, NET -255.18
2018-04-24, 1512.4065192114717
2018-04-24, Trade Number:2,CLOSING SHORT POSITION:9410.99
2018-04-25, BUY EXECUTED, Price: 9410.99, Cost: -756.20, Comm 0.00
2018-04-27, 636.6224539743337
2018-04-27, BUY LONG:9230.89
2018-04-28, BUY EXECUTED, Price: 9230.89, Cost: 636.62, Comm 0.00
2018-05-22, 9.999996564147295e-08
2018-05-22, Trade Number:3,CLOSING LONG POSITION:8214.39
2018-05-23, SELL EXECUTED, Price: 8214.39, Cost: 636.62, Comm 0.00
2018-05-23, OPERATION PROFIT, GROSS -70